
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# 2A - Data Ingestion with CREATE TABLE AS and COPY INTO

In this demonstration, we'll explore ingesting data from cloud storage into Delta tables with the `CREATE TABLE AS (CTAS)` AND `COPY INTO` statements.

### Learning Objectives

By the end of this lesson, you should be able to:

- Use the CTAS statement with `read_files()` to ingest Parquet files into a Delta table.
- Use `COPY INTO` to incrementally load Parquet files from cloud object storage into a Delta table.

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default and you have a Shared SQL warehouse.

<!-- ![Select Cluster](./Includes/images/selecting_cluster_info.png) -->

Follow these steps to select the classic compute cluster:


1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.

   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.


## A. Classroom Setup

Run the following cell to configure your working environment for this notebook.

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course in the lab environment.

In [0]:
%run ./Includes/Classroom-Setup-02

Run the cell below to view your default catalog and schema. Notice that your default catalog is **dbacademy** and your default schema is your unique **labuser** schema.

**NOTE:** The default catalog and schema are pre-configured for you to avoid the need to specify the three-level name when writing your tables to your **labuser** schema (i.e., catalog.schema.table).

In [0]:
SELECT current_catalog(), current_schema()

## B. Explore the Data Source Files


1. We'll create a table containing historical user data from Parquet files stored in the volume  
   `'/Volumes/dbacademy_ecommerce/v01/raw/users-historical'` within Unity Catalog.

   Use the `LIST` statement to view the files in this volume. Run the cell and review the results.

   Notice the files in the **name** column begin with **part-**. This shows that this volume contains multiple **Parquet** files.

In [0]:
LIST '/Volumes/dbacademy_ecommerce/v01/raw/users-historical'

2. Query the Parquet [files by path](https://docs.databricks.com/aws/en/query/#query-data-by-path) in the `/Volumes/dbacademy_ecommerce/v01/raw/users-historical` directory to view the raw data in tabular format to quickly preview the files.

In [0]:
SELECT * 
FROM parquet.`/Volumes/dbacademy_ecommerce/v01/raw/users-historical`;

## C. Batch Data Ingestion with CTAS and read_files()

The `CREATE TABLE AS` (CTAS) statement is used to create and populate tables using the results of a query. This allows you to create a table and load it with data in a single step, streamlining data ingestion workflows.

#### Automatic Schema Inference for Parquet Files

Apache Parquet is a columnar storage format optimized for analytical queries. It includes embedded schema metadata (e.g., column names and data types), which enables automatic schema inference when creating tables from Parquet files. This eliminates the need for manual schema definitions and simplifies the process of converting Parquet files into Delta format by leveraging the built-in schema metadata.

### C1. CTAS with the `read_files()` Function

The code in the next cell creates a table using CTAS with the `read_files()` function.

The `read_files()` table-valued function (TVF) enables reading a variety of file formats and provides additional options for data ingestion.

1. First, let's explore the documentation for `read_files`. Complete the following steps:

   a. Navigate to the [read_files](https://docs.databricks.com/aws/en/sql/language-manual/functions/read_files) documentation.

   b. Scroll down and find the **Options** section. Take a moment to explore some of the features of `read_files`.

   c. In the **Options** section, notice the variety of options available based on the file type.

   d. Click on **parquet** and scroll through the available options.

**NOTE:** The `read_files` function provides a wide range of capabilities and specific options for each file type. The previous method used to create a table only works if no additional options are required.


2. Use the `read_files()` function to query the same Parquet files located in `/Volumes/dbacademy_ecommerce/v01/raw/users-historical`. The `LIMIT` clause limits the amount of rows during exploration and development.

   - The first parameter in `read_files` is the path to the data.

   - The `format => "parquet"` option specifies the file format.

   The `read_files` function automatically detects the file format and infers a unified schema across all files. It also supports explicit schema definitions and `schemaHints`. For more details on schema inference capabilities, refer to the [Schema inference](https://docs.databricks.com/aws/en/sql/language-manual/functions/read_files#schema-inference) documentation.

**NOTE:** A **_rescued_data** column is automatically included by default to capture any data that doesn’t match the inferred schema.

In [0]:
SELECT * 
FROM read_files(
  '/Volumes/dbacademy_ecommerce/v01/raw/users-historical',
  format => 'parquet'
)
LIMIT 10;

3. Next, let's use `read_files()` with a CTAS statement to create the table **historical_users_bronze_ctas_rf**, then display the table.

   Notice that the Parquet files were ingested to create a table (Delta by default).

In [0]:
-- Drop the table if it exists for demonstration purposes
DROP TABLE IF EXISTS historical_users_bronze_ctas_rf;


-- Create the Delta table
CREATE TABLE historical_users_bronze_ctas_rf 
SELECT * 
FROM read_files(
        '/Volumes/dbacademy_ecommerce/v01/raw/users-historical',
        format => 'parquet'
      );


-- Preview the Delta table
SELECT * 
FROM historical_users_bronze_ctas_rf 
LIMIT 10;

4. Run the `DESCRIBE TABLE EXTENDED` statement to view column names, data types, and additional table metadata.  

   Review the results and notice the following:
   
   - The table was created in your schema within the course catalog **dbacademy**.

   - The *Type* row indicates that the table is *MANAGED*.

   - The *Location* row shows the managed cloud storage location.
   
   - The *Provider* row specifies that the table is a Delta table.


In [0]:
DESCRIBE TABLE EXTENDED historical_users_bronze_ctas_rf;

#### Managed vs External Tables in Databricks

##### Managed Tables
- Databricks **manages both the data and metadata**.
- Data is stored **within Databricks’ managed storage**.
- **Dropping the table also deletes the data**.
- Recommended for creating new tables.

##### External Tables
- Databricks **only manages the table metadata**.
- **Dropping the table does not delete the data**.
- Supports **multiple formats**, including Delta Lake.
- Ideal for **sharing data across platforms** or using existing external data.


### C2. (BONUS) Python Ingestion
The code uses Python to ingest the parquet files.

In [0]:
%python
# 1. Read the Parquet files from the volume into a Spark DataFrame
df = (spark
      .read
      .format("parquet")
      .load("/Volumes/dbacademy_ecommerce/v01/raw/users-historical")
    )


# 2. Write to the DataFrame to a Delta table (overwrite the table if it exists)
(df
 .write
 .mode("overwrite")
 .saveAsTable(f"dbacademy.{DA.schema_name}.historical_users_bronze_python")
)


## 3. Read and view the table
users_bronze_table = spark.table(f"dbacademy.{DA.schema_name}.historical_users_bronze_python")
users_bronze_table.display()

## D. Incremental Data Ingestion with `COPY INTO`
`COPY INTO` is a Databricks SQL command that allows you to load data from a file location into a Delta table. This operation is re-triable and idempotent, i.e. files in the source location that have already been loaded are skipped. This command is useful for when you need to load data into an existing Delta table. 

[COPY INTO](https://docs.databricks.com/aws/en/sql/language-manual/delta-copy-into)

### D1. Ingesting Parquet Files with COPY INTO

Using the same set of Parquet files as before, let's use `COPY INTO` to create our Bronze table again.

We will look at two examples:

1. Example 1: Common Schema Mismatch Error

1. Example 2: Preemptively Handling Schema Evolution

#### Example 1: Common Schema Mismatch Error

1. The cell below creates an empty table named **historical_users_bronze_ci** with a defined schema for only the **user_id** and **user_first_touch_timestamp** columns.

   However, the Parquet files in `'/Volumes/dbacademy_ecommerce/v01/raw/users-historical'` contain three columns: 
    - **user_id**
    - **user_first_touch_timestamp** 
    - **email**

   Run the cell below and review the error. You should see the `[COPY_INTO_SCHEMA_MISMATCH_WITH_TARGET_TABLE]` error. This error occurs because there is a schema mismatch: the Parquet files contain 3 columns, but the target table **historical_users_bronze_ci** only has 2 columns.

   How can you handle this error?

In [0]:
--------------------------------------------
-- This cell returns an error
--------------------------------------------

-- Drop the table if it exists for demonstration purposes
DROP TABLE IF EXISTS historical_users_bronze_ci;


-- Create an empty table with the specified table schema (only 2 out of the 3 columns)
CREATE TABLE historical_users_bronze_ci (
  user_id STRING,
  user_first_touch_timestamp BIGINT
);


-- Use COPY INTO to populate Delta table
COPY INTO historical_users_bronze_ci
  FROM '/Volumes/dbacademy_ecommerce/v01/raw/users-historical'
  FILEFORMAT = parquet;

2. We can fix this error by adding `COPY_OPTIONS` with the `mergeSchema = 'true'` option. When set to `true`, this option allows the schema to evolve based on the incoming data.

   Run the next cell with the `COPY_OPTIONS` option added. You should notice that the Parquet files were successfully ingested into the table, with a total of 251,501 rows ingested.

In [0]:
COPY INTO historical_users_bronze_ci
  FROM '/Volumes/dbacademy_ecommerce/v01/raw/users-historical'
  FILEFORMAT = parquet
  COPY_OPTIONS ('mergeSchema' = 'true');     -- Merge the schema of each file

3. Preview the data in the **historical_users_bronze_ci** table.

In [0]:
SELECT *
FROM historical_users_bronze_ci
LIMIT 10;

#### Example 2: Preemptively Handling Schema Evolution

1. Another way to ingest the same files into a Delta table is to start by creating an empty table named **historical_users_bronze_ci_no_schema**.

   Then, add the `COPY_OPTIONS ('mergeSchema' = 'true')` option to enable schema evolution for the table.

   Run the cell and confirm that 251,501 rows were added to the Delta table.

In [0]:
-- Drop the table if it exists for demonstration purposes
DROP TABLE IF EXISTS historical_users_bronze_ci_no_schema;


-- Create an empty table without the specified schema
CREATE TABLE historical_users_bronze_ci_no_schema;


-- Use COPY INTO to populate Delta table
COPY INTO historical_users_bronze_ci_no_schema
  FROM '/Volumes/dbacademy_ecommerce/v01/raw/users-historical'
  FILEFORMAT = parquet
  COPY_OPTIONS ('mergeSchema' = 'true');

### D2. Idempotency (Incremental Ingestion)

`COPY INTO` tracks the files it has previously ingested. If the command is run again, no additional data is ingested because the files in the source directory haven't changed.

1. Let's run the `COPY INTO` command again and check if any data is re-ingested into the table.

   Run the cell and view the results. Notice that the values for **num_affected_rows**, **num_inserted_rows**, and **num_skipped_corrupt_files** are all 0 because the data has already been ingested into the Delta table.

**NOTE**: If new files are added to the cloud storage location, `COPY INTO` will only ingest those files. Using `COPY INTO` is a great option if you want to run a job for incremental batch ingestion from cloud storage location without re-reading files that have already been loaded.

In [0]:
COPY INTO historical_users_bronze_ci_no_schema
  FROM '/Volumes/dbacademy_ecommerce/v01/raw/users-historical'
  FILEFORMAT = parquet
  COPY_OPTIONS ('mergeSchema' = 'true');

**NOTE:** For the remaining demos, we will not focus on `COPY INTO`. Instead, we will focus on leveraging the `read_files` function. As you gain experience you can begin to create streaming tables for incremental or streaming ingestion using SQL and DLT.

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>